In [1]:
import networkx as nx
import EoN
import matplotlib.pyplot as plt

In [ ]:
g = nx.watts_strogatz_graph(1000, 4, 0.3, seed=896803)
s = EoN.percolate_network(g, 0.8)

nx.draw(s ,  node_color=range(1000), node_size=5, edge_color='black', cmap=plt.cm.Reds_r, width= 0.5)
nx.draw(g ,  node_color=range(1000), node_size=5, edge_color='black', cmap=plt.cm.Reds_r, width= 0.5)

In [ ]:
%matplotlib inline 
import os, sys
import json

import random
import numpy as np
import networkx as nx
import pandas as pd

import matplotlib
from matplotlib import pyplot as plt

# Import any other package you may need
from pylab import *
from scipy.ndimage import measurements
import statistics 

In [ ]:
def visualize_grid(grid, params=None, figname=None):

    cmap = plt.cm.get_cmap('jet', len(np.unique(grid)))
    im = plt.matshow(grid, cmap=cmap, origin='lower', interpolation='none')
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.xticks([])
    plt.yticks([])
    
    if params != None:
        plt.title(params, fontsize=10)
    
    if figname != None:
        plt.savefig(figname, dpi=300, bbox_inches='tight')

def paint_grid(clusters,prevGrid):
    temp_grid = np.copy(prevGrid)
    for cluster in clusters:
      for coord in cluster:
        temp_grid[coord[0]][coord[1]] = 2.0
    return temp_grid

def create_grid(dimensions, pthr):
    
    grid = np.zeros(dimensions)
    
    for irow,i in enumerate(grid):
      for icolumn,j in enumerate(i):
        grid[irow][icolumn] =1 if pthr>random() else 0
    return grid

def find_clusters(grid):
    clusters = list()
    cGrids = np.zeros(grid.shape)
    lw, num = measurements.label(grid)   
    clusters = []
    for i in range(num+1):
      temp = []
      clusters.append(temp)
    
    labels = arange(lw.max() + 1)
    shuffle(labels)
    
    cGrids = labels[lw]
    for irow,i in enumerate(grid):
      for icolumn,j in enumerate(i):
        cGrids[irow][icolumn] = cGrids[irow][icolumn] if grid[irow][icolumn]!=0 else 0
        if (lw[irow][icolumn]>0):
          clusters[cGrids[irow][icolumn]].append((irow,icolumn))
    return cGrids,sorted(clusters, key=len)


def check_percolation(grid, clusters):
    clusters = sorted(clusters, key=len)
    percCluster = []
    temp_grid = np.zeros(grid.shape)
    # search for percolated cluster(s)
    for cluster in list(sorted(clusters,key=len,reverse=False)):
      
      left = []
      right = []
      up = []
      down = []
      
      for coord in cluster:
        if (coord[0]==0):
          up.append(coord)
        elif (coord[1]==0):
          left.append(coord)
        elif (coord[0]==grid.shape[0]-1):
          down.append(coord)
        elif (coord[1]==grid.shape[0]-1):
          right.append(coord)
        # if any cluster connected to open sites from up to down or left to right is found 
        # current cluster percolates
        if (len(down)>0 and len(up)>0):
          percCluster.append(cluster)
          break
        elif (len(left)>0 and len(right)>0):
          percCluster.append(cluster)
          break
    # if any percolating cluster(s) found, paint_grid paints them to grid and returns them 
    if (len(percCluster)>0):
      return paint_grid(percCluster,grid), percCluster
    return temp_grid, None

pval = 0.60
dims = (100,100)
rgrid = create_grid(dims, pval)
visualize_grid(rgrid, {'p':pval, 'dims':dims})

cgrid,clusters = find_clusters(rgrid)
visualize_grid(cgrid, {'p':pval, 'dims':dims})

pgrid, pcluster = check_percolation(rgrid, clusters)
visualize_grid(pgrid, {'p':pval, 'dims':dims})

In [ ]:
from pylab import *
from scipy.ndimage import measurements
import numpy as np
import matplotlib.pyplot as plt
import time
import sys
import signal

In [ ]:

def signal_handler(signal, frame):
    sys.exit(0)
signal.signal(signal.SIGINT, signal_handler)
 
# Set up the figures

fig1 = plt.figure(1, figsize=(7,4))
plt.title("Number of clusters")

fig2 = plt.figure(2, figsize = (7,4))
plt.title("Largest Cluster")
plt.ion()

time.sleep(20)

for L in [50, 100, 200, 500, 1000]: 
    for marker in ['ko', 'bo', 'ro']:
        r = rand(L,L)
        for p in np.arange(0., 1., 0.05):
            z = r<p

            lw, num = measurements.label(z)
            area = measurements.sum(z, lw, index=arange(lw.max() + 1))
            areaImg = area[lw]
            

            # Plot the number of clusters		
            plt.figure(1)
            plt.title("Number of clusters")
            plt.plot(p, lw.max(), marker)
            plt.draw()

            # Plot the size of the largest cluster		
            plt.figure(2)
            plt.title("Largest Cluster")
            plt.plot(p, area.max(), marker)
            plt.draw()
            
    plt.ioff()
    plt.show()    

In [ ]:
def signal_handler(signal, frame):
    sys.exit(0)
signal.signal(signal.SIGINT, signal_handler)
 
# Set up the figures


time.sleep(20)

for L in [50, 100, 200, 500, 1000]: 
    for marker in ['ko', 'bo', 'ro']:
		# Randomise the percolation network
        r = rand(L,L)
        for p in np.arange(0.2, 1., 0.02):
			# Determine the connectivity for probability p
            z = r<p

			# Define the clusters using the ndimage.measurements routines
            lw, num = measurements.label(z)

			# Plot the clusters for given p
            fig0 = plt.figure(0, figsize=(9,9))
            p1 = int(p* 1000)
            p1 = p1 / 1000
            plt.title("Clusters by area" + " , " + "p = " +str(p1))
            area = measurements.sum(z, lw, index=arange(lw.max() + 1))
            areaImg = area[lw]
            plt.imshow(areaImg, origin='lower', interpolation='nearest', vmin=1, vmax=L*5, cmap= plt.cm.Reds_r)
            plt.draw()
            fig0.savefig(fname= "Clusters by area" + str(p1) + " " + str(L)+ '.png', )
            plt.show()
